In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm

In [2]:
from architect import Architect
from model_search import Network

In [3]:
tf.enable_eager_execution()

In [4]:
def parse_args():
    parser = argparse.ArgumentParser("cifar")
    parser.add_argument('--data', type=str, default='./data',
                        help='location of the data corpus')
    parser.add_argument('--batch_size', type=int,
                        default=1, help='batch size')
    parser.add_argument('--learning_rate', type=float,
                        default=0.025, help='init learning rate')
    parser.add_argument('--learning_rate_min', type=float,
                        default=0.001, help='min learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
    parser.add_argument('--weight_decay', type=float,
                        default=3e-4, help='weight decay')
    parser.add_argument('--report_freq', type=float,
                        default=10, help='report frequency')
    parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
    parser.add_argument('--epochs', type=int, default=50,
                        help='num of training epochs')
    parser.add_argument('--init_channels', type=int,
                        default=3, help='num of init channels')
    parser.add_argument('--layers', type=int, default=5,
                        help='total number of layers')
    parser.add_argument('--model_path', type=str,
                        default='saved_models', help='path to save the model')
    parser.add_argument('--cutout', action='store_true',
                        default=False, help='use cutout')
    parser.add_argument('--cutout_length', type=int,
                        default=16, help='cutout length')
    parser.add_argument('--drop_path_prob', type=float,
                        default=0.3, help='drop path probability')
    parser.add_argument('--save', type=str, default='EXP',
                        help='experiment name')
    parser.add_argument('--seed', type=int, default=2, help='random seed')
    parser.add_argument('--grad_clip', type=float,
                        default=5, help='gradient clipping')
    parser.add_argument('--train_portion', type=float,
                        default=1.0, help='portion of training data')
    parser.add_argument('--unrolled', action='store_true',
                        default=False, help='use one-step unrolled validation loss')
    parser.add_argument('--arch_learning_rate', type=float,
                        default=3e-4, help='learning rate for arch encoding')
    parser.add_argument('--arch_weight_decay', type=float,
                        default=1e-3, help='weight decay for arch encoding')
    args = parser.parse_args()

    return args

# Train Function

In [5]:
def train(x_train, y_train, x_valid, y_valid, model, architect, optimizer):
    """Trains the network. Gradient step is performed here

    Args:
        train_queue (array): Train queue
        valid_queue (array): Validation queue
        model (Network): Network
        architect (Architect): the architechture of network
        criterion (fn): Loss function
        optimizer (Optimiser): Adam / SGD
        lr (float): Learning Rate

    Returns:
        (float, float): returns acc and miOu
    """

    # architect step
    architect_step = architect.step(input_train=x_train,
                                    target_train=y_train,
                                    input_valid=x_valid,
                                    target_valid=y_valid,
                                    unrolled=args.unrolled,
                                    )
    
    with tf.control_dependencies([architect_step]):
        with tf.GradientTape() as tape:
            logits = model(x_train)
            w_var = model.get_thetas()
            loss = model._criterion(logits, y_train)
            grads = tape.gradient(loss, w_var)
            clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
            opt_op = optimizer.apply_gradients(zip(clipped_gradients, w_var))

    # calculating accuracy and iou
    acc = utils.accuracy(logits, y_train)
    iou = utils.iou(logits, y_train)

    return loss, acc, iou

## Train Test

In [5]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "arch_weight_decay": 1e-3,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate_min": 0.001,
    "learning_rate": 0.025,
    "unrolled": True,
    "epochs": 10,
    "batch_size": 4,
    "save": "EXP"
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

In [6]:
np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(4)
ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(4)

In [7]:
criterion = tf.losses.sigmoid_cross_entropy
model = Network(3, 3, criterion)
optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)

W0926 14:12:10.865862 139687464281920 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:290: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0926 14:12:10.873477 139687464281920 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:7: The name tf.layers.MaxPooling2D is deprecated. Please use tf.compat.v1.layers.MaxPooling2D instead.

W0926 14:12:10.921126 139687464281920 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/model_search.py:97: The name tf.layers.Conv2DTranspose is deprecated. Please use tf.compat.v1.layers.Conv2DTranspose instead.



In [8]:
architect = Architect(model, args)

W0926 14:12:12.999419 139687464281920 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/architect.py:39: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



In [10]:
init = tf.global_variables_initializer()

In [37]:
for e in range(2):
    train_it = ds_train.make_one_shot_iterator()
    valid_it = ds_valid.make_one_shot_iterator()
    for i in tqdm(range(1)):
        x_train, y_train = train_it.get_next()
        x_valid, y_valid = valid_it.get_next()
        
        loss, acc_op, iou_op = train(x_train,
                                   y_train,
                                   x_valid,
                                   y_valid,
                                   model,
                                   architect,
                                   optimizer
                                  )




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:01<00:00,  1.13s/it]



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


# Infer

In [6]:
def infer(x_valid, y_valid, logits, model, criterion):
    loss_op = model._loss(logits, y_valid)
    acc_op = utils.accuracy(logits, y_valid)
    iou_op = utils.iou(logits, y_valid)
    return loss_op, acc_op, iou_op

# Main

In [9]:
def main(args):
    np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
    np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
    ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(args.batch_size)
    ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(args.batch_size)

    num_iterations = int(np_ds_train[0].shape[0] / args.batch_size)

    criterion = tf.losses.sigmoid_cross_entropy
    model = Network(3, 3, criterion)

    # Optimizer
    optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)

    architect = Architect(model, args)

    init = tf.global_variables_initializer()
    
    _logits = model(tf.convert_to_tensor(np_ds_train[0][:4]))
    mious = []
    for e in range(args.epochs):
        tf.logging.info('Epoch {}'.format(e))

        train_it = ds_train.make_one_shot_iterator()
        valid_it = ds_valid.make_one_shot_iterator()

        tq1 = tqdm(range(num_iterations))
        genotype = model.genotype()

        # Train Loop
        train_miou = 0
        train_unions, train_intersections = [], []    

        for i in tq1:
            x_train, y_train = train_it.get_next()
            x_valid, y_valid = valid_it.get_next()
            train_loss, train_acc, train_iou = train(x_train=x_train,
                                                     y_train=y_train,
                                                     x_valid=x_valid,
                                                     y_valid=y_valid,
                                                     model=model,
                                                     architect=architect,
                                                     optimizer=optimizer
                                                     )
            if(i % args.report_freq == 0):
                tq1.set_postfix({
                        "Train Loss": train_loss.numpy(),
                        "Train Acc": train_acc.numpy(),
                        "Train IoU": train_iou[0].numpy()
                        })
            train_intersections.append(train_iou[1].numpy())
            train_unions.append(train_iou[2].numpy())
    
        # Calculation of train miou
        train_unions = np.array(train_unions)
        train_intersections = np.array(train_intersections)
        train_non_zero_mask = train_unions != 0
        train_miou = np.mean(train_intersections[train_non_zero_mask])/(np.mean(train_unions[train_non_zero_mask]) + 1e-6)

        # Log train miou
        logging.info('Train mIoU: {}'.format(train_miou))

        # Validation loop
        valid_unions, valid_intersections = [], []
        tq2 = tqdm(range(num_iterations))
        for i in tq2:
            valid_logits = model(x_valid)
            valid_loss, valid_acc, valid_iou = infer(x_valid=x_valid, 
                                                       y_valid=y_valid,
                                                       logits=valid_logits,
                                                       model=model,
                                                       criterion=criterion
                                                       )

            if(i % args.report_freq == 0):
                tq2.set_postfix({
                                "Valid Loss": valid_loss.numpy(),
                                "Valid Acc": valid_acc.numpy(),
                                "Valid IoU": valid_iou[0].numpy()
                                })
            valid_intersections.append(valid_iou[1])  
            valid_unions.append(valid_iou[2])

        # Calculation of train miou
        valid_unions = np.array(valid_unions)
        valid_intersections = np.array(valid_intersections)
        valid_non_zero_mask = valid_unions != 0
        valid_miou = np.mean(valid_intersections[valid_non_zero_mask])/(np.mean(valid_unions[valid_non_zero_mask]) + 1e-6)

        #Log Miou
        logging.info('Validation mIoU: {}'.format(valid_miou))

        # save the final genotype
        if(max(mIoUs) < valid_iou):
            logging.info("Writing the computed genotype tp ./cnn/final_models/final_genotype.py")
            utils.write_genotype(genotype)

        mious.append(valid_miou)
  
    np.save(os.path.join(args.save, "mIoUs.npy"), mIoUs)

## Test

In [10]:
def parse_args():
    args = {
        "momentum": 0.9,
        "weight_decay": 3e-4,
        "arch_learning_rate": 3e-1,
        "arch_weight_decay": 1e-3,
        "momentum": 0.9,
        "grad_clip": 5,
        "learning_rate_min": 0.001,
        "learning_rate": 0.025,
        "unrolled": True,
        "epochs": 1,
        "batch_size": 4,
        "save": "EXP",
        "report_freq": 1
    }

    class Struct:
        def __init__(self, **entries):
            self.__dict__.update(entries)

    return Struct(**args)

In [11]:
if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.ERROR)
    args = parse_args()
    args.save = 'search-{}-{}'.format(args.save,
                                      time.strftime("%Y%m%d-%H%M%S"))
    utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))
    
    # logging
    log_format = '%(asctime)s %(message)s'
    logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                        format=log_format, datefmt='%m/%d %I:%M:%S %p')
    fh = logging.FileHandler(os.path.join(args.save, 'log.txt'))
    fh.setFormatter(logging.Formatter(log_format))
    logging.getLogger().addHandler(fh)
    

#     main(args)

Experiment dir : search-EXP-20190926-180242
